In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from transformator.tree import Tree
from transformator.lambda_tree import LambdaTree
import pyAgrum.lib.notebook as gnb

In [53]:
N = 6

In [54]:
ns = list(range(N-4, N+1))

In [51]:
classes = [list(LambdaTree.generate(k)) for k in ns]

In [52]:
[len(c) for c in classes]

[2, 4, 9, 22, 57]

In [134]:
# n-4:
# l^2*s^2*n*B(n)
#
# n-3:
# ((4*a*o*s*u - 2*l^2*s - 2*l*s^2)*(n - 1) + 6*a*o*s*u - 2*l^2*s - 3*l*s^2)*B(n)
# ((4*n+2)*a*o*s*u - 2*n*l^2*s - (2*n+1)*l*s^2)*B(n)
#
# n-2:
# ((-4*a*o*u + l^2 + 4*l*s + s^2)*(n - 2) - 8*u*o*a + 2*l^2 + 10*l*s + 3*s^2)*B(n)
# ((-4*n)*a*o*u + l^2*n + (4*n+2)*l*s + (n+1)*s^2)*B(n)
#
# n-1:
# ((-2*l - 2*s)*(n - 3) - 7*l - 8*s)*B(n)
# ((-2*n - 1)*l + (-2*n-2)*s)*B(n)
#
# n:
# (1 + n)*B(n)

r_ns = [
    ns[0] * len(classes[0]),
    ((4*ns[1]+2) - (2*ns[1]) - (2*ns[1]+1)) * len(classes[1]),
    (-4*ns[2] + ns[2] + (4*ns[2]+2) + (ns[2]+1)) * len(classes[2]),
    ((-2*ns[3]-1) + (-2*ns[3]-2)) * len(classes[3]),
    (ns[4] + 1) * len(classes[4]),
]

r_ns

[4, 4, 99, -506, 399]

In [135]:
sum(r_ns)

0

In [114]:
r_classes = [[[], []] for _ in range(5)]

In [137]:
# n:
# (1 + n)*B(n)

i = 4
r_classes[i] = [[], []]

r_classes[i][0] += sum(
    [
        list(
            t.visit_subtrees(
                lambda kind, left, right:
                    None if kind[0] is not None
                    else [kind + ("*", )] + left + right
            )
        )
        for t
        in classes[i]
    ],
    []
)

p = len(r_classes[i][0])
q = len(r_classes[i][1])
print(f"{p} - {q} = {p - q}")

gnb.sideBySide(*[t._repr_svg_() for t in r_classes[i][0][:5]])
gnb.sideBySide(*[t._repr_svg_() for t in r_classes[i][1][:5]])

399 - 0 = 399


""


In [141]:
# n-1:
# ((-2*n - 1)*l + (-2*n-2)*s)*B(n)

i = 3
r_classes[i] = [[], []]

r_classes[i][1] += sum(
    [
        list(
            t.visit_subtrees(
                lambda kind, left, right:
                    ["l", kind + ("*", )] + left + right + [None]
            )
        )
        for t
        in classes[i]
    ],
    []
)

r_classes[i][1] += sum(
    [
        list(
            t.visit_subtrees(
                lambda kind, left, right:
                    None if kind[0] is not None
                    else ["s", kind + ("*", ), None]
            )
        )
        for t
        in classes[i]
    ],
    []
)

r_classes[i][1] += sum(
    [
        list(
            t.visit_subtrees(
                lambda kind, left, right:
                    None if kind[0] is not None
                    else ["s", None, kind + ("*", )]
            )
        )
        for t
        in classes[i]
    ],
    []
)

p = len(r_classes[i][0])
q = len(r_classes[i][1])
print(f"{p} - {q} = {p - q}")

gnb.sideBySide(*[t._repr_svg_() for t in r_classes[i][0][:5]])
gnb.sideBySide(*[t._repr_svg_() for t in r_classes[i][1][:5]])

0 - 506 = -506


""


In [ ]:
# n-2:
# ((-4*n)*a*o*u + l^2*n + (4*n+2)*l*s + (n+1)*s^2)*B(n)



In [ ]:
# n-3:
# ((4*n+2)*a*o*s*u - 2*n*l^2*s - (2*n+1)*l*s^2)*B(n)


In [142]:
# n-4:
# l^2*s^2*n*B(n)

i = 0
r_classes[i] = [[], []]

r_classes[i][0] += sum(
    [
        list(
            t.visit_subtrees(
                lambda kind, left, right:
                    None if kind[0] is None
                    else ["l", "l", "s", None, "s", None, kind + ("*", )] + left + right + [None, None]
            )
        )
        for t
        in classes[i]
    ],
    []
)

p = len(r_classes[i][0])
q = len(r_classes[i][1])
print(f"{p} - {q} = {p - q}")

gnb.sideBySide(*[t._repr_svg_() for t in r_classes[i][0][:5]])
gnb.sideBySide(*[t._repr_svg_() for t in r_classes[i][1][:5]])

4 - 0 = 4


""


In [101]:
gnb.sideBySide(*[t._repr_svg_() for t in r_classes[0][0]])

In [102]:
gnb.sideBySide(*[t._repr_svg_() for t in r_classes[4][0][:5]])

In [165]:
r_classes[0][0]

[<l, l, s, , s, , l*, o, , , , , >,
 <l, l, l, s, , s, , o*, , , , , >,
 <l, l, s, , s, , s*, , o, , , , >,
 <s, , l, l, s, , s, , o*, , , , >]

True

(('l',),
 ('l',),
 ('s',),
 (None,),
 ('s',),
 (None,),
 ('l', '*'),
 ('o',),
 (None,),
 (None,),
 (None,),
 (None,),
 (None,))